<a href="https://colab.research.google.com/github/sakamrani2020/Image_Classification_VGG16_Model/blob/main/IMAGE_CLASSIFICATION_VGG16_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMAGE_CLASSIFICATION_USING_VGG16_MODEL**

**CIFAR-10 Dataset:** The dataset consists of 60,000 32x32 color images in 10 classes, with each class having 6,000 images. The images are divided into a training set of 50,000 images and a test set of 10,000 images.
The **ten classes** in the CIFAR-10 dataset are:
**1)Airplane 2)Automobile 3)Bird 4)Cat 5)Deer 6)Dog 7)Frog 8)Horse 9)Ship 10)Truck**

In [ ]:
!pip install tensorflow keras pillow pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.12.0-py3-none-any.whl size=213956 sha256=902a228026cfbfd2c068a237c6ca554ece44f85542dd4b64c97aa28f91cf639a
  Stored in directory: /root/.cache/pip/wheels/7e/ba/82/f3ab5bc48525778633bccc741c0424677ed3435736221819f4
Successfully built pyTelegramBotAPI


In [ ]:
import os
import numpy as np
import telebot
import requests
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import load_model
from PIL import Image

# Telegram Bot Token
BOT_TOKEN = '6332189721:AAFi-5P81lXb1vFENBaIV5HxkqEtP-tybFs'  # Replace with your Telegram bot token
bot = telebot.TeleBot(BOT_TOKEN)

# Load the CIFAR-10 dataset
(_, _), (test_images, test_labels) = cifar10.load_data()
test_labels = to_categorical(test_labels)

# Load the VGG16 model
vgg16_model = VGG16(weights='imagenet')

# Function to preprocess the images for VGG16
def preprocess_image_vgg16(image):
    img = Image.open(image)
    img = img.resize((224, 224))
    img = np.array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

# Function to predict the class of an image using VGG16
def predict_image_class_vgg16(image):
    img = preprocess_image_vgg16(image)
    predictions = vgg16_model.predict(img)
    decoded_predictions = decode_predictions(predictions, top=1)[0]
    class_name = decoded_predictions[0][1]
    return class_name

# Command to handle incoming images
@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    try:
        # Download the image sent by the user
        file_id = message.photo[-1].file_id
        file_info = bot.get_file(file_id)
        image_url = f"https://api.telegram.org/file/bot{BOT_TOKEN}/{file_info.file_path}"
        image_name = f"{file_id}.jpg"

        # Download the image
        image_path = os.path.join(".", image_name)
        image_content = requests.get(image_url).content
        with open(image_path, "wb") as f:
            f.write(image_content)

        # Predict the image class using VGG16
        class_name = predict_image_class_vgg16(image_path)

        # Reply to the user with the predicted class name
        reply_message = f"The image contains the class: {class_name}."
        bot.send_message(message.chat.id, reply_message)

        # Remove the downloaded image after processing
        os.remove(image_path)

    except Exception as e:
        bot.reply_to(message, f"Error occurred: {e}")

# Command to start the bot and show help message
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    help_message = "Welcome to the Image Classification Bot!\n\n" \
                   "Please send an image to classify its contents."
    bot.reply_to(message, help_message)

if __name__ == "__main__":
    bot.polling()
